# Interactive Introduction to ML and AI with a RAG-System

Based on a PDF containing a starter set of DND 5e character [sheets](https://dnd5echaractersheet.com/)


## sys admin

Create a .env file with the following content:

`OPENAI_API_KEY = "^<API_KEY>"`

In [1]:
# required libraries for the tutorial
import openai
import os
from dotenv import load_dotenv
from langchain.document_loaders import PyPDFLoader
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

## Load PDF data
Loads the data and splits it into chunks.
Each chunk contains 1000 characters max with a max overlap of 100 characters.

In [8]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
loader = PyPDFLoader("data/Personalrecht.pdf")
chunks = loader.load_and_split(text_splitter)

### Check the chunks
get chunk content with: chunks[index].page_content

In [9]:
print(chunks[0])
print("The chunk contains " + str(len(chunks[0].page_content)) + " characters")

page_content='177.100 \n177.101 \n \n \n \n  \nPersonalrecht\nVerordnung\nüber das Arbeitsverhältnis\ndes städtischen Personals und\nAusführungsbestimmungen\nGültig ab 1. Juli 2002\nNachgeführt bis 7. Februar 2007' metadata={'source': 'data/Personalrecht.pdf', 'page': 0}
The chunk contains 187 characters


## Setup models

We need to prepare an embedding model to vectorise our chunks before storing them into our ChromaDB and a language model to generate answers to our questions.

In [10]:

# Load environment variables from .env file
load_dotenv()

# Access the API key using the variable name defined in the .env file
openai.api_key = os.getenv("OPENAI_API_KEY")

# Initialize the OpenAI chat model
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.8)

# initialize the OpenAI embeddings model
embeddings = OpenAIEmbeddings()

### Load / Create Chroma DB

We check for the existence of the directory for 2 reasons:
1) We use Openai Embeddings and pay for the embedding generation
2) Chroma does not overwrite an existing database, but allows to upate it

In [11]:
if os.path.exists("chroma"):
    print("Loading Chroma from disk...")
    Chroma(persist_directory="chroma", embedding_function=embeddings)
else:
    chroma_db = Chroma.from_documents(documents=chunks,
                                    embedding=embeddings,
                                    persist_directory="chroma",
                                    collection_name="lc_chroma_demo")

### Test Your Database

In [17]:
query = "Was ist der Ferienanspruch?"

Simple Similarity Search

In [18]:
result = chroma_db.similarity_search(query)
print(result)

[Document(page_content='Anspruch auf eine zusätzliche Ferienwoche haben, jedoch nur, soweit der Saldo die wöchentliche Normalarbeitszeit gemäss Art. 157 Abs. 1 und 2 übersteigt.', metadata={'page': 88, 'source': 'data/Personalrecht.pdf'}), Document(page_content='7Die Ferien sind bis Ende April de s Folgejahres zu beziehen. Ist \ndies nicht möglich, kann die Dienstchefin oder der Dienstchef den späteren Bezug bewilligen.', metadata={'page': 105, 'source': 'data/Personalrecht.pdf'}), Document(page_content='Ausführungsbestimmungen zum Personalrecht  \n 79 Art. 118 Ferienanspruch bei Stundenlohn \nDer Ferienanspruch von Angeste llten im Stundenlohn wird un-\nter Vorbehalt von Art. 117 Abs. 2 tageweise wie folgt berechnet: \na) bei vier Wochen Ferien im J ahr: ein Ferientag auf 109 Ar-\nbeitsstunden; \nb) bei fünf Wochen Ferien im Jahr: ein Ferientag auf 87 Ar-\nbeitsstunden; \nc) bei sechs Wochen Ferien im Jahr: ein Ferientag auf 72 Ar-\nbeitsstunden. \nArt. 119 Bezug der Ferien \n1Der grö

Similarity Search with Scores

In [19]:
result_with_scores = chroma_db.similarity_search_with_score(query)
print(result_with_scores)

[(Document(page_content='Anspruch auf eine zusätzliche Ferienwoche haben, jedoch nur, soweit der Saldo die wöchentliche Normalarbeitszeit gemäss Art. 157 Abs. 1 und 2 übersteigt.', metadata={'page': 88, 'source': 'data/Personalrecht.pdf'}), 0.28654855489730835), (Document(page_content='7Die Ferien sind bis Ende April de s Folgejahres zu beziehen. Ist \ndies nicht möglich, kann die Dienstchefin oder der Dienstchef den späteren Bezug bewilligen.', metadata={'page': 105, 'source': 'data/Personalrecht.pdf'}), 0.3275854289531708), (Document(page_content='Ausführungsbestimmungen zum Personalrecht  \n 79 Art. 118 Ferienanspruch bei Stundenlohn \nDer Ferienanspruch von Angeste llten im Stundenlohn wird un-\nter Vorbehalt von Art. 117 Abs. 2 tageweise wie folgt berechnet: \na) bei vier Wochen Ferien im J ahr: ein Ferientag auf 109 Ar-\nbeitsstunden; \nb) bei fünf Wochen Ferien im Jahr: ein Ferientag auf 87 Ar-\nbeitsstunden; \nc) bei sechs Wochen Ferien im Jahr: ein Ferientag auf 72 Ar-\nbeitss

In [20]:
chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=chroma_db.as_retriever())

In [21]:
response = chain.invoke(query)
print(response)

{'query': 'Was ist der Ferienanspruch?', 'result': 'Der Ferienanspruch bezieht sich auf die Anzahl der Urlaubstage, die Arbeitnehmer gemäß den geltenden Bestimmungen und Arbeitsverträgen erhalten. In diesem Fall wird der Ferienanspruch anhand der wöchentlichen Normalarbeitszeit und der Anzahl der Wochen Urlaub pro Jahr berechnet. Es wird auch darauf hingewiesen, dass der Großteil der Ferien zusammenhängend genommen werden sollte, sofern betrieblich möglich.'}


### Test some queries Yourself

In [43]:
def get_response(query:str):
    ## add the functionality to combine the functionalities above.
    chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=chroma_db.as_retriever())
    response = chain.invoke(query)
    print(response)

get_response("")    

    

{'query': 'Anspruch auf was?', 'result': 'Die Angestellten haben Anspruch auf regelmäßige Beurteilung von Leistung und Verhalten sowie auf ein Arbeitszeugnis, das über die Art und die Dauer des Arbeitsverhältnisses sowie über ihre Leistungen und ihr Verhalten Auskunft gibt.'}



In [49]:
previous_responses = []

In [45]:

def get_response(query: str):
    # Assuming RetrievalQA is a class that you've defined
    chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=chroma_db.as_retriever())

    # Append the current query to the list of previous responses
    previous_responses.append(query)

    # Concatenate the previous responses into a single string
    context = " ".join(previous_responses)

    # Invoke the retrieval question-answering chain with the concatenated context
    response = chain.invoke(context)
    print("Response:", response)

# Example usage


In [48]:
get_response("Wer ist Präsident der USA?")


Response: {'query': 'Wie lange dauert der Mutterschaftsurlaub Und was ist das? Wer ist Präsident der USA?', 'result': 'Der Mutterschaftsurlaub dauert insgesamt 16 Kalenderwochen und ist ein bezahlter Urlaub, den eine Angestellte vor und nach der Geburt ihres Kindes nehmen kann. Zum Zeitpunkt der letzten Aktualisierung dieser Informationen war Joe Biden der Präsident der USA.'}
